In [34]:
function fornberg_weights(z,x,n,m)
    # From Bengt Fornbergs (1998) SIAM Review paper.
    #  	Input Parameters
    #	z location where approximations are to be accurate,
    #	x(0:nd) grid point locations, found in x(0:n)
    #	n one less than total number of grid points; n must
    #	not exceed the parameter nd below,
    #	nd dimension of x- and c-arrays in calling program
    #	x(0:nd) and c(0:nd,0:m), respectively,
    #	m highest derivative for which weights are sought,
    #	Output Parameter
    #	c(0:nd,0:m) weights at grid locations x(0:n) for derivatives
    #	of order 0:m, found in c(0:n,0:m)
    #      	dimension x(0:nd),c(0:nd,0:m)
    
      c = zeros(n+1,m+1);
      c1 = 1.0;
      c4 = x[1+0]-z;
      for k=0:m
            for j=0:n
                  c[1+j,1+k] = 0.0;
            end
      end
      c[1+0,1+0] = 1.0;
      for  i=1:n
            mn = min(i,m);
            c2 = 1.0;
            c5 = c4;
            c4 = x[1+i]-z;
            for j=0:i-1
                  c3 = x[1+i]-x[1+j];
                  c2 = c2*c3;
                    if (j == i-1)
                        for k=mn:-1:1
                        c[1+i,1+k] = c1*(k*c[1+i-1,1+k-1]-c5*c[1+i-1,1+k])/c2;
                        end
                  c[1+i,1+0] = -c1*c5*c[1+i-1,1+0]/c2;
                  end
                  for k=mn:-1:1
                        c[1+j,1+k] = (c4*c[1+j,1+k]-k*c[1+j,1+k-1])/c3;
                  end
                  c[1+j,1+0] = c4*c[1+j,1+0]/c3;
            end
            c1 = c2;
      end
      return c
end # weights function    

fornberg_weights (generic function with 1 method)

In [35]:
function stencil(x::AbstractVector{<:Real}, x₀::Real, m::Integer)
    ℓ = 0:length(x)-1
    m in ℓ || throw(ArgumentError("order $m ∉ $ℓ"))
    A = @. (x' - x₀)^ℓ / factorial(ℓ)
    return A \ (ℓ .== m) # vector of weights w
end

stencil (generic function with 2 methods)

In [36]:
using BenchmarkTools

In [37]:
# Define the grid points and the target point
x = [-3, -2, -1, 0, 1, 2, 3]
n = length(x) - 1 # definition of n
x₀ = 0 # evaluate a derivative at this point 
m = 2  # First derivative

# Compute the finite difference weights
@btime test1 = fornberg_weights(x₀, x, n, m) # gives weights for ALL derivatives from 0 to m given 
@btime test2 = stencil(x, x₀, m)

println("Finite difference weights v1: ", test1[:, end])
println("Finite difference weights v2: ", test2)

  170.149 ns (1 allocation: 224 bytes)
  1.054 μs (6 allocations: 1.19 KiB)
Finite difference weights v1: [0.011, -0.15, 1.5, -2.7, 1.5, -0.15, 0.011]
Finite difference weights v2: [0.011, -0.15, 1.5, -2.7, 1.5, -0.15, 0.011]


So the first implementation (which is closer to Fornberg's original algorithm) is MUCH faster. Good to know!